# Answering Business Questions using SQL

Quick Links: [Tables and Views](#Tables-and-Views) | [Defining the Business Problem](#Defining-the-Business-Problem)

----

<p style='text-align: justify;'>
For this project, we'll be working with a modified version of a database called <b>Chinook</b>. The Chinook database represents a fictitious digital media store or music shop, including tables for artists, albums, media tracks, invoices and customers.
<br><br>
The <a href='https://github.com/lerocha/chinook-database'>Chinook database</a> contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. Here's a schema diagram for the Chinook database:
</p>
<img src='https://s3.amazonaws.com/dq-content/191/chinook-schema.svg'/ title='Chinook Schema'>
<p style='font-size: 75%; text-align: center'><b>Fig. 1 Schema Diagram for the Chinook Database</b></p>

<p style='text-align: justify;'>
Tables names are shown in bold, with the columns in each table listed below it. Each table has one or more columns with shading, which indicates those columns are a <b>primary key</b>.
<br><br>
Relationships between tables are shown using <b>lines</b> between the tables. The lines indicate which columns are related. You may notice that at least one 'end' of the relationship will be a primary key.
</p>

Let's begin by connecting our notebook to the database file.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### Tables and Views
We can also quert a list of all the tables and views in our database to get more familiar with our data.

In [2]:
%%sql
SELECT
      name,
      type
FROM sqlite_master
WHERE type IN ("table", "view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


<p style='text-align: justify;'>
As we saw in our scheme diagram, there are indeed eleven tables. Let's try joining the <code>customer</code> and <code>invoice</code> tables by the former's primary key, the <code>customer_id</code> column.
</p>

In [3]:
%%sql
   SELECT
          customer.*,
          invoice.*
     FROM customer
LEFT JOIN invoice
       ON customer.customer_id = invoice.customer_id
    LIMIT 5;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id,invoice_id,customer_id_1,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3,16,1,2017-01-26 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,8.91
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3,77,1,2017-05-28 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,5.9399999999999995
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3,149,1,2017-11-30 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,8.91
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3,153,1,2017-12-14 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,13.86
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3,182,1,2018-02-07 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,5.9399999999999995


<p style='text-align: justify;'>
As we can see, there are instances in <code>invoice</code> where a unique <code>customer_id</code> occurs more than once, hence our output table having (at least) 5 rows for Luis Gonçalves.
<br><br>
Let's now proceed to answering business questions that we can answer through SQL queries.
</p>

### Defining the Business Problem

#### 1. Adding Albums by Genre Sellability

<p style='text-align: justify;'>
The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:
</p>

|     Artist Name    | Genre |
|--------------------|-------|
|Regal               |Hip-Hop|
|Red Tone            |Punk   |
|Meteor and the Girls|Pop    |
|Slim Jim Bites      |Blues  |

<p style='text-align: justify;'>
The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in <b>finding out which genres sell the best in the USA</b>.
</p>

In [4]:
%%sql

/*Get Tracks Sold in USA*/
WITH
    usa_invoices AS 
    (
        SELECT il.*
          FROM invoice_line il
    INNER JOIN invoice i
               ON il.invoice_id = i.invoice_id
    INNER JOIN customer c
               ON i.customer_id = c.customer_id
         WHERE c.country = "USA"
    )
 
/*Aggregate Tracks Sold by Genre*/
    SELECT g.name Genre,
           COUNT(ui.quantity) `Tracks Sold`,
           ROUND(
                 CAST(COUNT(ui.quantity) AS FLOAT)/(
                 SELECT COUNT(*) FROM usa_invoices)
                 , 2)*100 `Perc Tracks Sold`
      FROM usa_invoices ui
INNER JOIN track t
           ON ui.track_id = t.track_id
INNER JOIN genre g
           ON t.genre_id = g.genre_id
  GROUP BY Genre
  ORDER BY `Tracks Sold` DESC;

Done.


Genre,Tracks Sold,Perc Tracks Sold
Rock,561,53.0
Alternative & Punk,130,12.0
Metal,124,12.0
R&B/Soul,53,5.0
Blues,36,3.0
Alternative,35,3.0
Latin,22,2.0
Pop,22,2.0
Hip Hop/Rap,20,2.0
Jazz,14,1.0


<p style='text-align: justify'>
Looking at the above result, the albums/artist tracks in our list that we should add to the store based on tracks sold are:
</p>

1. <b>Red Tone</b> (Alternative & Punk as a genre is the second top-selling)
2. <b>Slim Jim Blues</b> (A few ways lower than the above is Blues)
3. <b>Meteor and the Girls</b> (Also lower is Pop)

<p style='text-align: justify'>
<b>Regal</b> unfortunately belongs in the Hip-Hop genre group which isn't selling as much as the other three. It is worth noting that to actually capitalize on what is selling, we should look for artists who produce <b>Rock</b> music as it comprised <code>53%</code> of all tracks sold while the aforementioned three combined account for only <code>18%</code>.
</p>

#### 2. Comparing Employee Performance

<p style='text-align: justify'>
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.
</p>

In [5]:
%%sql

    SELECT e.first_name || " " || e.last_name `Employee Name`,
           SUBSTR(e.birthdate, 1, 10) `Employee Birthdate`,
           SUBSTR(e.hire_date, 1, 10) `Employee Hire Date`, 
           c.first_name || " " || c.last_name `Customer Name`,
           ROUND(SUM(i.total), 2) `Total Sales`
      FROM employee e
INNER JOIN customer c
           ON e.employee_id = c.support_rep_id
INNER JOIN invoice i
           ON c.customer_id = i.customer_id
  GROUP BY e.employee_id
  ORDER BY `Total Sales` DESC;

Done.


Employee Name,Employee Birthdate,Employee Hire Date,Customer Name,Total Sales
Jane Peacock,1973-08-29,2017-04-01,Phil Hughes,1731.51
Margaret Park,1947-09-19,2017-05-03,Dan Miller,1584.0
Steve Johnson,1965-03-03,2017-10-17,Mark Philips,1393.92


<p style='text-align: justify;'>
If purely in terms of sales, <b>Jane performs the best</b> while <b>Steve performs the worst</b>. However, looking at the hiring date of all three, there is a <b>positive correlation</b> between years employed and performance in that Jane has been employed the longest while Steve is the most recently hired among the three. 
<br><br>
Thus, the <b>difference in sales</b> may be due to the employee having more or less experience and/or opportunity to  be assigned a customer. A more worthwhile approach of measuring performance should include enough data points in date intervals shared by all employees.
</p>

#### 3. Analysis of Sales by Country

<p style='text-align: justify;'>
Our next task is to analyze the sales data for customers from each different country. We make use of the country value from the <code>customers</code> table, and ignore the country from the billing address in the <code>invoice</code> table.
<br><br>
In particular, for each country, we get:
</p>

- Total Number of Customers
- Total Value of Sales
- Average Value of Sales per Customer
- Average Order Value

<p style='text-align: justify;'>
Because there are a number of countries with only one customer, we group these customers as "Others" in our analysis.
</p>

In [6]:
%%sql

/*Assign "Other" as Country for Those that have Only One Customer*/
WITH
    pre_count AS
        (
           SELECT country,
                  COUNT(customer_id) num_of_customers
             FROM customer
         GROUP BY country
        ),
    
    customer_new AS
        (
            SELECT *,
                   CASE
                       WHEN pre_count.num_of_customers = 1 THEN "Others"
                       ELSE customer.country
                       END AS country_new
              FROM customer
         LEFT JOIN pre_count 
                   ON customer.country = pre_count.country  
        )

/*Aggregate by Country*/
    SELECT cn.country_new Country,
           COUNT(DISTINCT cn.customer_id) `Total Customers`,
           ROUND(SUM(il.unit_price), 2) `Total Sales`,
           ROUND(SUM(il.unit_price)/COUNT(DISTINCT cn.customer_id), 2) `Ave. Sales per Customer`,
           ROUND(SUM(il.unit_price)/COUNT(DISTINCT il.invoice_id), 2) `Ave. Order Value`
      FROM (
            SELECT *,
                   CASE
                       WHEN country_new = "Others" THEN 1
                       ELSE 0
                       END AS sort
            FROM customer_new
           ) cn
INNER JOIN invoice i
        ON cn.customer_id = i.customer_id
INNER JOIN invoice_line il
        ON i.invoice_id = il.invoice_id
  GROUP BY cn.country_new
  ORDER BY sort ASC, `Total Sales` DESC, `Ave. Order Value` DESC;

Done.


Country,Total Customers,Total Sales,Ave. Sales per Customer,Ave. Order Value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Others,15,1094.94,73.0,7.45


<p style='text-align: justify;'>
Before we proceed further into the analysis, it is worth noting that because of the small sample size for each country, it is highly likely that the averages (and hence the totals) will be easily <b>skewed</b> and become <b>unrealiable</b>. Thus, proposed marketing campaigns borne out of this analysis should be relatively conservative in spending until enough samples are collected.
<br><br>
Looking at the above table and noting the above, the countries where it is worth investing in in terms of order value are:
</p>

1. Czech Republic (Also note the high average sale price, but the small sample size of two)
2. United Kingdom (similarly small sample)
3. India (similarly small sample)

#### 3. Individual Track Purchases vs. Album Purchases

<p style='text-align: justify;'>
The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:
</p>

- purchase a whole album
- purchase a collection of one or more individual tracks.

<p style='text-align: justify;'>
The store does <b>not</b> let customers purchase a whole album, <b>and then</b> add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.
<br><br>
Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase <b>only the most popular tracks</b> from each album from record companies, instead of purchasing every track from an album.
<br><br>
We have been asked to find out what percentage of purchases are individual tracks vs. whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.
</p>

In [7]:
%%sql

WITH
    album_new AS
        (
            SELECT an.album_id,
                   an.title album_title,
                   t.name track_name,
                   t.track_id
              FROM album an
         LEFT JOIN track t
                   ON an.album_id = t.album_id
        ),
        
    invoice_line_new AS
        (
         SELECT invoice_id,
                track_id
           FROM invoice_line
        ),
        
    invoice_final AS
        (
             SELECT * 
               FROM invoice_line_new iln
         INNER JOIN album_new an
                    ON iln.track_id = an.track_id
        ),
        
    category_final AS
        (
         SELECT
               if.invoice_id,
               CASE
                   WHEN (
                         SELECT track_id FROM invoice_final WHERE invoice_final.invoice_id = if.invoice_id
                         EXCEPT
                         SELECT track_id FROM album_new WHERE album_new.album_id = if.album_id
                        ) IS NULL
                   AND  (
                         SELECT track_id FROM album_new WHERE album_new.album_id = if.album_id
                         EXCEPT
                         SELECT track_id FROM invoice_final WHERE invoice_final.invoice_id = if.invoice_id
                        ) IS NULL
                  THEN "Album"
                  ELSE "Individual"
                  END AS Category
         FROM invoice_final if
         GROUP BY if.invoice_id
        )


SELECT Category `Purchase Category`,
       COUNT(invoice_id) `Number of Invoices`,
       ROUND(
             CAST(COUNT(invoice_id) AS FLOAT)/(SELECT COUNT(*) FROM category_final)
             , 4)*100 `Percent of Invoices`
FROM category_final
GROUP BY Category;

Done.


Purchase Category,Number of Invoices,Percent of Invoices
Album,114,18.57
Individual,500,81.43


<p style='text-align: justify;'>
Based on the result of the above query, invoices that involve purchasing the whole album account for <code>18.57%</code> of all invoices. Despite majority of the invoices being of purchases of individual tracks, the potential loss in revenue when we consider purchasing only the most popular tracks from each album from record companies is still considerable. Thus, it may still be worth purchasing whole albums.
</p>